In [1]:
# PAPER - dELVING dEEP INTO RECTIFIERS:  SURPASSING HUMAN-LEVEL PERFORMANCE ON IMAGENET CLASSIFICATION BY KAIMING HE

In [2]:
# PAPER - BATCH NORMALIZATION: ACCELERATING DEEP NETWORK TRAINING BY REDUCING INTERNAL COVARIATION SHIFT BY TEAM AT GOOGLE - 2015

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [4]:
# reading all the words
words = open('cleaned_names.txt', 'r').read().splitlines()
words[:8]

['aaban', 'aabid', 'aabidah', 'aabir', 'aabriella', 'aada', 'aadam', 'aadarsh']

In [5]:
len(words)

29681

In [6]:
# building the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [7]:
# building the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):
  X, Y = [], []

  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80% of the data
Xdev, Ydev = build_dataset(words[n1:n2]) # 10% of the data
Xte, Yte = build_dataset(words[n2:]) # 10% of the data

torch.Size([170379, 3]) torch.Size([170379])
torch.Size([21222, 3]) torch.Size([21222])
torch.Size([21124, 3]) torch.Size([21124])


In [8]:
# MLP revisited
n_embd = 10 # the dimensionality of the embeddings
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd),          generator=g) # the embedding matrix
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * ((5/3) / (n_embd * block_size)**0.5) # 0.2 # the first layer of the MLP # "Kaiming init"
# b1 = torch.randn((n_hidden),                    generator=g) * 0.01 # the bias of the first layer
W2 = torch.randn((n_hidden, vocab_size),        generator=g) * 0.01 # the second layer of the MLP
b2 = torch.randn((vocab_size),                  generator=g) * 0 # the bias of the second layer

bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias] # b1
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True


12097


In [9]:
(torch.randn(10000) * 0.2).std() # numbers for gaussian

tensor(0.1982)

In [10]:
n_embd * block_size

30

In [11]:
(5/3) / (30**0.5)

0.3042903097250923

In [ ]:
x = torch.randn(1000, 10)
w = torch.randn(10, 200) / 10**0.5
y = x @ w
print(x.mean(), x.std())
print(y.mean(), y.std())


tensor(-0.0065) tensor(1.0038)
tensor(-0.0005) tensor(1.0111)


In [13]:
# x = torch.randn(1000, 10)
# w = torch.randn(10, 200)
# y = x @ w
# print(x.mean(), x.std())
# print(y.mean(), y.std())
# plt.figure(figsize=(16,4))
# plt.subplot(121)
# plt.hist(x.view(-1.tolist(), 50, density=True));
# plt.subplot(122)
# plt.hist(y.view(-1).tolist(), 50, density=True)

In [14]:
# same optimization as in part 2
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

  # minibatch construction
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors

  # Linear Layer
  hpreact = embcat @ W1 # + b1 # hidden layer pre-activation

  # BatchNorm Layer
  # -----------------------------------------------------------------------
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)
  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias # normalizing the batches by mini-batch mean, mini-batch variance, and scale & shift from the batch normalization paper

  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 + bnmeani
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
  # ------------------------------------------------------------------------

  # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias # normalizing the batches by mini-batch mean, mini-batch variance, and scale & shift from the batch normalization paper

  # Non-Linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function

  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # tracking the stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: loss = {loss.item():.4f}')
  lossi.append(loss.log10().item())

  break

      0/ 200000: loss = 3.2934


In [15]:
hpreact.shape

torch.Size([32, 200])

In [16]:
hpreact.mean(0, keepdim=True).shape # mini batch mean

torch.Size([1, 200])

In [17]:
hpreact.std(0, keepdim=True).shape # mini-batch variance

torch.Size([1, 200])

In [18]:
# plt.figure(figsize=(20,10))
# plt.imshow(h.abs() > 0.99, cmap='gray', interpolation='nearest')

In [19]:
h

tensor([[ 0.5102, -0.8575, -0.9743,  ...,  0.9134, -0.4221, -0.9897],
        [ 0.6293, -0.6538, -0.4349,  ..., -0.5975, -0.5392,  0.6251],
        [-0.8935, -0.3233, -0.8561,  ...,  0.1991, -0.4329,  0.1723],
        ...,
        [-0.6813,  0.6284, -0.6896,  ..., -0.8029,  0.5645,  0.0265],
        [-0.8849,  0.7388,  0.6854,  ...,  0.1415,  0.6940,  0.8267],
        [ 0.0597,  0.1482, -0.5840,  ...,  0.7168,  0.7829, -0.8592]],
       grad_fn=<TanhBackward0>)

In [20]:
h.shape

torch.Size([32, 200])

In [21]:
# plt.hist(h.view(-1).tolist(), 50);

In [22]:
# plt.hist(hpreact.view(-1).tolist(), 50);

In [23]:
logits[0] # logits should be much much closer to zero

tensor([-1.0592e-02,  7.5373e-02, -1.1673e-01, -1.2082e-01, -8.0378e-02,
        -1.1445e-01, -5.8866e-02, -1.0877e-01, -1.4605e-01,  1.5571e-01,
        -2.1340e-01,  2.0722e-04, -1.2685e-02,  4.9772e-02, -8.3668e-02,
         1.2821e-01,  8.3311e-02,  1.8042e-01,  1.9492e-02,  5.9680e-02,
        -3.9921e-02, -7.9148e-02, -9.2317e-02,  4.5156e-02,  3.6649e-03,
         2.2516e-02,  1.2429e-02], grad_fn=<SelectBackward0>)

In [24]:
# probability for any character
-torch.tensor(1/27.0).log()

tensor(3.2958)

In [25]:
# 4 dimensional example of the issue
logits = torch.tensor([1.0, 1.0, 1.0, 1.0])
# logits = torch.randn(4) * 10
probs = torch.softmax(logits, dim=0)
loss = -probs[2].log()
logits, probs, loss

(tensor([1., 1., 1., 1.]),
 tensor([0.2500, 0.2500, 0.2500, 0.2500]),
 tensor(1.3863))

In [26]:
# plt.plot(lossi)

In [27]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtr]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 #+ b1

  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnstd = hpreact.std(0, keepdim=True)

In [28]:
@torch.no_grad() # this ensures that we don't save gradients
def split_loss(split):
  x,y = {
    'train': (Xtr, Ytr),
    'val': (Xdev, Ydev),
    'test': (Xte, Yte)
  }[split]
  emb = C[x] # (N, block_size, n_embd) embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors (N, block_size * n_embd)
  h = torch.tanh(embcat @ W1) #+ b1) # hidden layer (N, n_hidden)

  hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
  hpreact = bngain * (hpreact -bnmean_running) / bnstd_running + bnbias 

  # hpreact = bngain * (hpreact - hpreact.mean(0, keepdim=True)) / hpreact.std(0, keepdim=True) + bnbias # normalizing the batches by mini-batch mean, mini-batch variance, and scale & shift from the batch normalization paper
  logits = h @ W2 + b2 # output layer (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())
  
split_loss('train')
split_loss('val')
split_loss('test')

train 3.2724852561950684
val 3.273871421813965
test 3.272106170654297


In [29]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10) # for reproducibility

for _ in range(20):

  out = []
  context = [0] * block_size # initialize with all dots
  while True:
    # forward pass
    emb = C[torch.tensor([context])] # embedding
    h = torch.tanh(emb.view(1, -1) @ W1) #+ b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    probs = F.softmax(logits, dim=1) # probabilities
    
    # sample from the distribution
    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    
    # shift the context window and track the sample
    context = context[1:] + [ix]
    out.append(ix)
    
    # stop if we reach the end
    if ix == 0:
      break

  print(''.join(itos[i] for i in out)) # decode and print the sample

erbujgmyazzhzelvndhfyalwqetvrsjejdrljg.
azejedoeliihsmpqpparekepxvqowxarveptzcmhvqrpcn.
szdbvrgxhgmjest.
jcjrwjkjqqxntpucfkubzmgvdcrxyqqufmehg.
kmejqvskkydmzhjl.
cakyybsufpzbjglvjurenjcrtvwsvevoeczpfhdxjfqyxoeobhggpcixjsfcueywhkhmrjolviilqiif.
evpkxwvlla.
odtabfsztpndgoikvsbmitgfkxvbssivlu.
meewrkepxcmvyhxavttudkjgczekhgcgtipvtudw.
iwbdnihzvorlq.
cjakepxugniagcflctfmiipzjgvonxet.
jydnblkskzporygnogsvmwosfzybsyknymjcxtkncjbmu.
etew.
aow.
copmrlxkzdcqbcep.
ccdfivsnicimhtecmokfgouwzlrurdckappomtcokwxtnbnzkdphfam.
xcojfffuqcxzhosy.
sj.
ascnpc.
cilytanczhrdtebimm.


In [30]:
# Training a Deeper Network in Pytorch

class Linear:

  def __init__(self, fan_in, fan_out, bias=True):
    self.weight = torch.randn((fan_in, fan_out), generator = g) / fan_in**0.5
    self.bias = torch.zeros(fan_out) if bias else None

  def __call__(self, x):
    self.out = x @ self.weight
    if self.bias is not None:
      self.out += self.bias
    return self.out
  
  def parameters(self):
    return [self.weight] + ([] if self.bias is None else [self.bias])
  
class BatchNorm1d:

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.momentum = momentum
    self.training = True

    # parameters (trained with backprop)
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

    # buffers (trained with a running 'momentum update')
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    # calculate the forward pass
    if self.training:
      xmean = x.mean(0, keepdim=True) # batch mean
      xvar = x.var(0, keepdim=True, unbiased= True) # batch variancce
    else:
      xmean = self.running_mean
      xvar = self.running_var
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta

    # update the buffers
    if self.training:
      with torch.no_grad():
        self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
        self.running_var  = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out
  
  def parameters(self):
    return [self.gamma, self.beta]
  
class Tanh:
  def __call__(self, x):
    self.out = torch.tanh(x)
    return self.out
  def parameters(self):
    return
  
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 100 # the number of the neurons in the hidden layer of MLP
g = torch.Generator().manual_seed(2147483467) # for reproducbility

C = torch.randn((vocab_size, n_embd),         generator=g)
layers = [
  Linear(n_embd * block_size, n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), Tanh(),
  Linear(           n_hidden, n_hidden), Tanh(),
  Linear(           n_hidden, vocab_size),
]

with torch.no_grad():
  # Last Layer: make less confident
  layers[-1].weight *= 0.1
  # all other layers: apply gain
  for layer in layers[:-1]:
    if isinstance(layer, Linear):
      layer.weight *= 5/3

parameters = [C] + [p for layer in layers if hasattr(layer, "parameters") and layer.parameters() is not None for p in layer.parameters()]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

46497


In [31]:
# same optimization as in part 2
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    # minibatch construction
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]  # batch X,Y

    # forward pass
    emb = C[Xb]  # embed the characters into vectors
    x = emb.view(emb.shape[0], -1)  # concatenate the vectors (initialize x)

    for layer in layers:
        x = layer(x)  # forward pass through each layer

    loss = F.cross_entropy(x, Yb)  # compute the loss

    # backward pass
    for layer in layers:
        layer.out.retain_grad()  # retain gradient for debugging (optional)
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01  # step learning rate decay
    for p in parameters:
        p.data += -lr * p.grad

    # tracking the stats
    if i % 10000 == 0:  # print every once in a while
        print(f'{i:7d}/{max_steps:7d}: loss = {loss.item():.4f}')
    lossi.append(loss.log10().item())

    break  # AFTER_DEBUG: would take out obviously to run full optimization


      0/ 200000: loss = 3.3120


In [32]:
# visualizing histograms
# plt.figure(figsize=(20, 4)) # width and height of the plot
# legends = []
# for i, p in enumerate(layer[:-1]): # ignore the last layer
#   if isinstance(layer, Tanh):
#     t = layer.out
#     print('layer %d (%10s): min = %.3f, max = %.3f' % (i, type(layer).__name__, t.min().item(), t.max().item()))
#     legends.append('%.3f .. %.3f' % (t.min().item(), t.max().item()))
# 
#   plt.subplot(1, len(layers), i+1)
#   plt.title(f'{type(layer).__name__} {i+1}')
#   plt.hist(layer.out.view(-1).tolist(), 50)

In [ ]:
# visualizing histograms for backward pass
# plt.figure(figsize=(20, 4)) # width and height of the plot
# legends = []
# for i, p in enumerate(layer[:-1]): # ignore the last layer
#   if isinstance(layer, Tanh):
#     t = layer.out.grad
#     print('layer %d (%10s): min = %.3f, max = %.3f' % (i, type(layer).__name__, t.min().item(), t.max().item()))
#     legends.append('%.3f .. %.3f' % (t.min().item(), t.max().item()))
# 
#   plt.subplot(1, len(layers), i+1)
#   plt.title(f'{type(layer).__name__} {i+1}')
#   plt.hist(layer.out.view(-1).tolist(), 50)